In [1]:
import requests
import time
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
# initialize list to store scraped data 
main_list =[]

# function to extract an html page 
def extract(url): 
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
    r= requests.get(url, headers = headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup.find_all('div', class_='search-listing')

In [11]:
# function to get business name, address, phone number, and rating for each page
def get_details(search_page):
    
    for search_listing in search_page:
        # get rating of each business
        rating = search_listing.find ('div', class_ = 'search-rating-container').text.strip().replace('\n','')#.strip() funtion to remove unnecessary end lines
        
            
        # REG EX Application
        if re.findall("^[1-5]", rating): # Only get businesses with 1 - 5 rating 
            business_name = search_listing.find ('h2', class_='search-tradename').text.strip().replace('\n','') 
            address = search_listing.find ('div', class_='search-busines-address').text.strip().replace('\n','')
            try:
                phone_number = search_listing.find ('a', class_ = 'btn btn-yp-default mr-2 biz-btn-call yp-click')['data-phone']
            except:
                phone_number =''
                    
            business_url = "https://www.yellow-pages.ph"+ search_listing.find("a", class_='yp-click')['href'] +"/reviews"
            
            # Extract Review page of business
            try:
                headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
                r= requests.get(business_url, verify=False)
                soup = BeautifulSoup(r.content, 'html.parser')
                review_page = soup.find_all('div', class_='yp-review-container')
                for content in review_page:
                    review= content.find('p', class_= None).text
    
            except:
                review = " "
            
            # store values in a dictionary and to the main_list[] list   
            business = {
                'Business name' : business_name,
                'Address': address,
                'Phone Number': phone_number,
                'Rating': rating,
                'Url' : business_url,
                'Review': review
                }
            main_list.append(business)
                    
        else:
            print ("no rating found on business")
      
    return
        

In [12]:
def load():
    df = pd.DataFrame(main_list)
    df.to_csv('Manila Businesses Most Rated.csv', index=False)

for x in range(1,9):
    print(f'Getting page {x}')
    search_page = extract(f'https://www.yellow-pages.ph/search/business/metro-manila/page-{x}?sort=most-rated')
    get_details(search_page)
    time.sleep(5)
    
load()
print('Saved to CSV')

Getting page 1
Getting page 2
Getting page 3
Getting page 4
Getting page 5
Getting page 6
Getting page 7
Getting page 8
Saved to CSV


In [7]:
del main_list